<a href="https://colab.research.google.com/github/krchnst/Ecommerce-AB-Test-Pipeline/blob/main/ecommerce_ab_test_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import auth
from google.cloud import bigquery

In [ ]:
auth.authenticate_user()
client = bigquery.Client(project="data-analytics-mate")

In [ ]:
query = """
WITH
session_info AS (
 SELECT
   s.date,
   s.ga_session_id,
   sp.country,
   sp.device,
   sp.continent,
   sp.channel,
   ab.test,
   ab.test_group
 FROM `DA.ab_test` ab
 JOIN `DA.session` s
   ON ab.ga_session_id = s.ga_session_id
 JOIN `DA.session_params` sp
   ON sp.ga_session_id = s.ga_session_id
),


session_with_orders AS (
 SELECT
   si.date,
   si.country,
   si.device,
   si.continent,
   si.channel,
   si.test,
   si.test_group,
   COUNT(DISTINCT o.ga_session_id) AS session_with_orders
 FROM `DA.order` o
 JOIN session_info si
   ON si.ga_session_id = o.ga_session_id
 GROUP BY
   si.date, si.country, si.device, si.continent, si.channel, si.test, si.test_group
),


events AS (
 SELECT
   si.date,
   si.country,
   si.device,
   si.continent,
   si.channel,
   si.test,
   si.test_group,
   ep.event_name,
   COUNT(ep.ga_session_id) AS event_cnt
 FROM `DA.event_params` ep
 JOIN session_info si
   ON ep.ga_session_id = si.ga_session_id
 GROUP BY
   si.date, si.country, si.device, si.continent, si.channel, si.test, si.test_group, ep.event_name
),


sessions_agg AS (
 SELECT
   si.date,
   si.country,
   si.device,
   si.continent,
   si.channel,
   si.test,
   si.test_group,
   COUNT(DISTINCT si.ga_session_id) AS session_cnt
 FROM session_info si
 GROUP BY
   si.date, si.country, si.device, si.continent, si.channel, si.test, si.test_group
),


accounts_agg AS (
 SELECT
   si.date,
   si.country,
   si.device,
   si.continent,
   si.channel,
   si.test,
   si.test_group,
   COUNT(DISTINCT acs.ga_session_id) AS new_account_cnt
 FROM `DA.account_session` acs
 JOIN session_info si
   ON acs.ga_session_id = si.ga_session_id
 GROUP BY
   si.date, si.country, si.device, si.continent, si.channel, si.test, si.test_group
)


-- Фінал: long-формат
SELECT
 swo.date,
 swo.country,
 swo.device,
 swo.continent,
 swo.channel,
 swo.test,
 swo.test_group,
 'session with orders' AS event_name,
 swo.session_with_orders AS value
FROM session_with_orders AS swo


UNION ALL
SELECT
 e.date,
 e.country,
 e.device,
 e.continent,
 e.channel,
 e.test,
 e.test_group,
 e.event_name,
 e.event_cnt AS value
FROM events AS e


UNION ALL
SELECT
 s.date,
 s.country,
 s.device,
 s.continent,
 s.channel,
 s.test,
 s.test_group,
 'session cnt' AS event_name,
 s.session_cnt AS value
FROM sessions_agg AS s


UNION ALL
SELECT
 a.date,
 a.country,
 a.device,
 a.continent,
 a.channel,
 a.test,
 a.test_group,
 'new account' AS event_name,
 a.new_account_cnt AS value
FROM accounts_agg AS a;
"""

query_job = client.query(query)  # Виконання SQL-запиту
results = query_job.result()  # Очікування завершення запиту

In [ ]:
df = results.to_dataframe()
display(df)

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-02,Paraguay,desktop,Americas,Organic Search,2,1,session with orders,1
1,2020-11-03,Ecuador,mobile,Americas,Direct,2,2,session with orders,1
2,2020-11-04,Slovenia,desktop,Europe,Organic Search,2,2,session with orders,1
3,2020-11-05,Malta,mobile,Europe,Paid Search,2,1,session with orders,1
4,2020-11-06,Croatia,mobile,Europe,Direct,2,2,session with orders,1
...,...,...,...,...,...,...,...,...,...
800991,2020-12-25,Vietnam,desktop,Asia,Paid Search,4,1,session_start,1
800992,2020-11-04,Vietnam,desktop,Asia,Paid Search,1,2,first_visit,1
800993,2020-12-06,Vietnam,mobile,Asia,Organic Search,3,1,first_visit,1
800994,2021-01-06,Vietnam,desktop,Asia,Organic Search,4,2,view_item,1


In [ ]:
df["event_name"].unique()

array(['session with orders', 'new account', 'session cnt', 'page_view',
       'view_promotion', 'user_engagement', 'scroll', 'view_item',
       'select_promotion', 'session_start', 'add_shipping_info',
       'first_visit', 'add_payment_info', 'begin_checkout', 'add_to_cart',
       'view_search_results', 'select_item', 'click', 'view_item_list'],
      dtype=object)

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# Вхідні дані
metrics = ["add_payment_info", "add_shipping_info", "begin_checkout", "new account"]
session_event = "session cnt"

# Групування подій
grouped = df[df["event_name"].isin(metrics + [session_event])].groupby(
    ["test", "test_group", "event_name"]
)["value"].sum().reset_index()

# Створення таблиці з метриками
pivoted = grouped[grouped["event_name"].isin(metrics)].pivot_table(
    index=["test", "event_name"], columns="test_group", values="value"
).reset_index()
pivoted.columns.name = None
pivoted.rename(columns={1: "numerator_cc", 2: "numerator_ev"}, inplace=True)

# Додавання сесій
sessions = grouped[grouped["event_name"] == session_event].pivot(
    index="test", columns="test_group", values="value"
).reset_index()
sessions.rename(columns={1: "denominator_cc", 2: "denominator_ev"}, inplace=True)

# Об'єднання метрик і сесій
merged = pivoted.merge(sessions, on="test")

# Перевірка, чи метрика реально присутня в тесті
available_metrics = grouped[grouped["event_name"].isin(metrics)][["test", "event_name"]].drop_duplicates()
merged = merged.merge(available_metrics, on=["test", "event_name"], how="inner")

# Розрахунок конверсій
merged["conversion_ev"] = merged["numerator_ev"] / merged["denominator_ev"]
merged["conversion_cc"] = merged["numerator_cc"] / merged["denominator_cc"]
merged["metric_change"] = merged["conversion_ev"] - merged["conversion_cc"]

# Функція для z-тесту
def z_test(row):
    x1, n1 = row["numerator_cc"], row["denominator_cc"]
    x2, n2 = row["numerator_ev"], row["denominator_ev"]
    p_pool = (x1 + x2) / (n1 + n2)
    se = np.sqrt(p_pool * (1 - p_pool) * (1/n1 + 1/n2))
    z = (row["conversion_ev"] - row["conversion_cc"]) / se if se > 0 else 0
    p = 2 * (1 - norm.cdf(abs(z)))
    return pd.Series([z, p, p < 0.05])

# Застосування тесту
merged[["z_stat", "p_value", "significant"]] = merged.apply(z_test, axis=1)

# Додавання назв подій
merged["numerator_ev_name"] = merged["event_name"]
merged["denominator_ev_name"] = "session"

# Формування фінальної таблиці
final = merged[[
    "test", "event_name",
    "numerator_ev_name", "denominator_ev_name",
    "numerator_ev", "denominator_ev", "conversion_ev",
    "numerator_cc", "denominator_cc", "conversion_cc",
    "metric_change", "z_stat", "p_value", "significant"
]]

# Перейменування колонок
final.columns = [
    "test_number", "metric",
    "numerator_ev_name", "denominator_ev_name",
    "numerator_ev", "denominator_ev", "conversion_ev",
    "numerator_cc", "denominator_cc", "conversion_cc",
    "metric_change", "z_stat", "p_value", "significant"
]

# Експорт у CSV
final.to_csv("ab_test_results.csv", index=False)


In [ ]:
from google.colab import files
files.download("ab_test_results.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

https://drive.google.com/file/d/16TaGza_gXdTTGqG9A9mLCAdfdQS4rzXi/view?usp=sharing

https://public.tableau.com/views/ABtest_17622965583910/ABtest?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link

This dashboard provides a comprehensive analysis of A/B test results across user segments and key conversion metrics. It highlights statistically significant changes using a two-sample z-test and allows filtering by test number, geography, channel, and device. The visualizations help identify which user actions were meaningfully impacted by the experiment.